In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Stacy's code starts here

In [47]:
# Load naturalization table
nat_file = 'fy2018_naturalization.xlsx'
filepath = os.path.join('.', 'Resources', nat_file)

naturalization = pd.read_excel(filepath, header=5)

In [48]:
# Drop notes data at end of document
naturalization.drop(labels=range(112,118), inplace=True)
naturalization.tail()

,Year,filed,Total,Civilian,Military 2,Not reported,denied
107,2014,773824.0,653416.0,642431.0,7468.0,3517,66767.0
108,2015,783062.0,730259.0,720645.0,7234.0,2380,75810.0
109,2016,972151.0,753060.0,742090.0,8885.0,2085,86033.0
110,2017,986851.0,707265.0,695718.0,6883.0,4664,83176.0
111,2018,810548.0,761901.0,750771.0,4495.0,6635,92586.0


In [49]:
# Rename columns
naturalization.rename(columns={
    'filed': 'Petitions filed',
    'Total': 'Naturalized, total',
    'denied': 'Petitions denied'
}, inplace=True)

# Drop unneeded columns
naturalization = naturalization.drop(labels=['Civilian', 'Military 2', 'Not reported'], axis='columns')

In [50]:
# Find funny/footnoted years and fix them
for index, row in naturalization.iterrows():
    
    year = naturalization.loc[index, 'Year']
    
    if len(str(year)) > 4:
        year = int(year[0:5])
        naturalization.loc[index, 'Year'] = year
    else:
        pass

# Set year as index
naturalization.set_index(keys=['Year'], inplace=True)

,Petitions filed,"Naturalized, total",Petitions denied
Year,,,
1907,21113.0,7941.0,250.0
1908,44032.0,25975.0,3330.0
1909,43141.0,38374.0,6341.0
1910,55750.0,39448.0,7781.0
1911,74740.0,56683.0,9017.0


In [51]:
# Change datatype to int for all columns
for c in naturalization.columns:
    naturalization[c] = naturalization[c].astype('int')

In [53]:
naturalization.head()

,Petitions filed,"Naturalized, total",Petitions denied
Year,,,
1907,21113,7941,250
1908,44032,25975,3330
1909,43141,38374,6341
1910,55750,39448,7781
1911,74740,56683,9017


In [ ]:
# Stacy's code ends here

In [ ]:
# Kana's code starts here

In [ ]:
# Kana's code ends here

In [ ]:
# Satish's code starts here

In [ ]:
# Satish's code ends here

In [ ]:
# Umar's code starts here

In [ ]:
# Umar's code ends here